# Data Science Library demo
Here I demonstrate the data science library I developed to quickly build scikit learn pipelines with optional scaling, feature interaction, data transformation (e.g. PCA, t-SNE) steps. It runs the pipeline through a grid-search (all combinations or a specific number of them) stratified (if classification) k-folds cross-validation and outputs the best model.

## Titanic dataset
Here I use the Titanic dataset I've cleaned and pickled in a separate tutorial.

### Import data

In [1]:
import pandas as pd

df = pd.read_pickle('trimmed_titanic_data.pkl')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890 entries, 0 to 890
Data columns (total 9 columns):
Survived    890 non-null int64
Pclass      890 non-null int64
Sex         890 non-null object
Age         890 non-null float64
SibSp       890 non-null int64
Parch       890 non-null int64
Fare        890 non-null float64
Embarked    890 non-null object
Title       890 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 69.5+ KB


By "cleaned" I mean I've derived titles (e.g. "Mr.", "Mrs.", "Dr.", etc) from the passenger names, imputed the missing Age values using polynomial regression with grid-searched 10-fold cross-validation, filled in the 3 missing Embarked values with the mode, and removed all fields that could be considered an id for that individual.

Thus, there is no missing data.

## Set categorical features as that type

In [2]:
simulation_df = df.copy()

categorical_features = ['Survived','Pclass','Sex','Embarked','Title']

for feature in categorical_features:
    simulation_df[feature] = simulation_df[feature].astype('category')
    
simulation_df.info()

# df["A"].astype('category')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890 entries, 0 to 890
Data columns (total 9 columns):
Survived    890 non-null category
Pclass      890 non-null category
Sex         890 non-null category
Age         890 non-null float64
SibSp       890 non-null int64
Parch       890 non-null int64
Fare        890 non-null float64
Embarked    890 non-null category
Title       890 non-null category
dtypes: category(5), float64(2), int64(2)
memory usage: 39.3 KB


## One-hot encode categorical features

In [3]:
simulation_df = pd.get_dummies(simulation_df,drop_first=True)

simulation_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890 entries, 0 to 890
Data columns (total 17 columns):
Age               890 non-null float64
SibSp             890 non-null int64
Parch             890 non-null int64
Fare              890 non-null float64
Survived_1        890 non-null uint8
Pclass_2          890 non-null uint8
Pclass_3          890 non-null uint8
Sex_male          890 non-null uint8
Embarked_Q        890 non-null uint8
Embarked_S        890 non-null uint8
Title_Dr          890 non-null uint8
Title_Military    890 non-null uint8
Title_Miss        890 non-null uint8
Title_Mr          890 non-null uint8
Title_Mrs         890 non-null uint8
Title_Noble       890 non-null uint8
Title_Rev         890 non-null uint8
dtypes: float64(2), int64(2), uint8(13)
memory usage: 46.1 KB


Now we have 17 features.

### Split into input/output data

In [4]:
# Set output feature
output_feature = 'Survived_1'

# Get all column names
column_names = list(simulation_df.columns)

# Exclude one of every categorical variable since the other one-hot encodings cover everything
input_features = [x for x in column_names if x != output_feature]

# Split into features and responses
X = simulation_df[input_features].copy()
y = simulation_df[output_feature].copy()

### Null model

In [5]:
simulation_df['Survived_1'].value_counts().values/float(simulation_df['Survived_1'].value_counts().values.sum())

array([ 0.61573034,  0.38426966])

Thus, null accuracy of ~62% if always predict death.

### Import data science library and initialize model collections

In [6]:
import data_science_lib as dsl

models = {}

/Users/cmshymansky/anaconda/lib/python2.7/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/Users/cmshymansky/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/cmshymansky/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  De

### Basic models w/ no pre-processing
#### KNN
Here I do a simple K-nearest neighbors (KNN) classification with 10-fold (default) cross-validation with a grid search over the default of 1 to 30 nearest neighbors and the use of either "uniform" or "distance" weights:

In [7]:
%%time 
reload(dsl)
        
# Figure out best model
models['knn'] = dsl.train_model(X,y,
                                use_default_param_dist=True,
                                random_state=6,
                                suppress_output=False, # Can suppress print outs if desired
                                estimator='knn',) 

Grid parameters:
estimator__n_neighbors : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
estimator__weights : ['uniform', 'distance']

Training set classification accuracy:  0.72893258427

Test set classification accuracy:  0.73595505618
Confusion matrix: 

 [[99  7]
 [40 32]]

Normalized confusion matrix: 

 [[ 0.55617978  0.03932584]
 [ 0.2247191   0.17977528]]

Classification report: 

              precision    recall  f1-score   support

          0       0.71      0.93      0.81       106
          1       0.82      0.44      0.58        72

avg / total       0.76      0.74      0.71       178


Best parameters:

{'estimator__n_neighbors': 12, 'estimator__weights': 'uniform'}

 Pipeline(steps=[('estimator', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=12, p=2,
           weights='uniform'))])
CPU times: user 1.86 s, sys: 108 ms, total: 

Turns out that the best settings are 30 neighbors and the use of the 'distance' weight.

Note how I've set the random_state to 6 so that the models can be compared using the same test/train split.

The output of the train_model() method is a Pipeline object with the optimal parameters found during the grid search and trained on all data.

Additional fields have been added to the pipeline object. These extra parameters are the type of score ('regression' or 'classification'), '.score_type', the training score (L2 norm for regression and classification accuracy for classification), '.train_score', the corresponding test score, '.test_score'. 

For classification problems, additional parameters also include the confusion matrix, '.confusion_matrix', normalized confusion matrix, '.normalized_confusion_matrix', and the classification report, '.classification_report'.

Here are the outputs of these additional parameters:

In [10]:
model_name = 'knn'

pipeline = models[model_name]

print 'Model:','knn','\n'
print 'Training score:\t\t', pipeline.train_score
print 'Test score:\t\t', pipeline.test_score,'\n'
print pipeline.classification_report

print 'Confusion matrix:\n\n',pipeline.confusion_matrix,'\n'
print 'Normalized confusion matrix:\n\n',pipeline.normalized_confusion_matrix,'\n'
print 'Best parameters:\n', pipeline.best_parameters

Model: knn 

Training score:		0.72893258427
Test score:		0.73595505618 

             precision    recall  f1-score   support

          0       0.71      0.93      0.81       106
          1       0.82      0.44      0.58        72

avg / total       0.76      0.74      0.71       178

Confusion matrix:

[[99  7]
 [40 32]] 

Normalized confusion matrix:

[[ 0.55617978  0.03932584]
 [ 0.2247191   0.17977528]] 

Best parameters:
{'estimator__n_neighbors': 12, 'estimator__weights': 'uniform'}


The print out from the solution above indicates that the default parameters to grid over are n_neighbors from 1 to 30 and the weights parameter as either 'uniform' or 'distance'.

This can be changed in two different ways. One way is to overwrite the parameter values by setting the param_dist keyword argument with the use_default_param_dist set to True:

In [ ]:
%%time 
reload(dsl)

# Set custom parameters
param_dist = {
    'estimator__n_neighbors': range(30,500)
}

# Figure out best model
models['custom_overwrite_knn'] = dsl.train_model(X,y,
                                       use_default_param_dist=True,
                                       random_state=6,
                                       suppress_output=False, # Can suppress print outs if desired
                                       estimator='knn',
                                       param_dist = param_dist) 

We were able to get a slightly better accuracy doing this.

The second way to use different parameter grid values is to set them with the custom param_dist keyword argument yet set use_default_param_dist to False. This makes it so that you must set every single parameter manually.

Here's an example:

In [ ]:
%%time 
reload(dsl)

# Set custom parameters
param_dist = {
    'estimator__n_neighbors': range(30,500)
}

# Figure out best model
models['from_scratch_knn'] = dsl.train_model(X,y,
                                       use_default_param_dist=False,
                                       random_state=6,
                                       suppress_output=False, # Can suppress print outs if desired
                                       estimator='knn',
                                       param_dist = param_dist) 

Note how the estimator__weights parameter isn't set for the KNN estimator.

#### Other models
This code currently supports K-nearest neighbors, logistic regression, support vector machines, multilayer perceptrons, random forest, and adaboost. 

We can loop through and pick the best model like this:

In [ ]:
%%time 
reload(dsl)

# Set model names to iterate over
model_names = ['knn','logistic_regression','svm','multilayer_perceptron','random_forest','adaboost']        

# Cross-validate each model
for model_name in model_names:
    models[model_name] = dsl.train_model(X,y,
                                    use_default_param_dist=True,
                                    random_state=6,
                                    estimator=model_name)

In [ ]:
trained_model_names = models.keys()

model_scores = [models[model].test_score for model in trained_model_names]

max_score = max(model_scores)

best_model = trained_model_names[model_scores.index(max_score)]

print best_model,max_score, '\n\n',models[best_model].classification_report

It turns out that the best model is logistic regression with a classfication accuracy of ~88%.

### Scaled data then classification
We can specify the scale_type keyword argument to scale the data before being fed to the desired estimator. Currently only standard scaling is supported:

In [ ]:
%%time 
reload(dsl)

# Set model names to iterate over
model_names = ['knn','logistic_regression','svm','multilayer_perceptron','random_forest','adaboost']        

# Cross-validate each model
for model_name in model_names:
    models['scaled_%s'%(model_name)] = dsl.train_model(X,y,
                                                       use_default_param_dist=True,
                                                       random_state=6,
                                                       scale_type = 'standard',
                                                       estimator=model_name)

In [ ]:
trained_model_names = models.keys()

model_scores = [models[model].test_score for model in trained_model_names]

max_score = max(model_scores)

best_model = trained_model_names[model_scores.index(max_score)]

print best_model,max_score, '\n\n',models[best_model].classification_report

### Feature selection, scaling, and classification
The feature_selection_type keyword argument can be used to select the best features:

In [ ]:
%%time 
reload(dsl)

# Set model names to iterate over
model_names = ['knn','logistic_regression','svm','multilayer_perceptron','random_forest','adaboost']        

# Cross-validate each model
for model_name in model_names:
    models['select_scaled_%s'%(model_name)] = dsl.train_model(X,y,
                                                       use_default_param_dist=True,
                                                       random_state=6,
                                                       scale_type = 'standard',
                                                       feature_selection_type = 'select_k_best', 
                                                       estimator=model_name)

In [ ]:
trained_model_names = models.keys()

model_scores = [models[model].test_score for model in trained_model_names]

max_score = max(model_scores)

best_model = trained_model_names[model_scores.index(max_score)]

print best_model,max_score, '\n\n',models[best_model].classification_report

It turns out that logistic regression without scaling outperforms all combinations of scalling and the classifiers.

### Scaled, transformed, then classification
Setting the transform_type keyword argument allows the data to be transformed into a new coordinate system that is dependent on the algorithm.

Currently, only principal component analysis (PCA) and t-distributed stochastic neighbor embedding (t-SNE) are supported.

#### PCA transformation

In [ ]:
%%time 
reload(dsl)

# Set model names to iterate over
model_names = ['knn','logistic_regression','svm','multilayer_perceptron','random_forest','adaboost']        

# Cross-validate each model
for model_name in model_names:
    models['scaled_pca_%s'%(model_name)] = dsl.train_model(X,y,use_default_param_dist=True,
                                                       random_state=6,
                                                        transform_type='pca',
                                                       scale_type = 'standard',
                                                       estimator=model_name)

In [ ]:
trained_model_names = models.keys()

model_scores = [models[model].test_score for model in trained_model_names]

max_score = max(model_scores)

best_model = trained_model_names[model_scores.index(max_score)]

print best_model,max_score,'\n\n',models[best_model].classification_report

Transformation with PCA doesn't appear to improve our results so far.

#### t-SNE

In [ ]:
%%time 
reload(dsl)

# Set model names to iterate over
model_names = ['knn','logistic_regression','svm','multilayer_perceptron','random_forest','adaboost']        

# Cross-validate each model
for model_name in model_names:
    models['scaled_t-sne_%s'%(model_name)] = dsl.train_model(X,y,use_default_param_dist=True,
                                                       random_state=6,
                                                        transform_type='t-sne',
                                                       scale_type = 'standard',
                                                       estimator=model_name)

In [ ]:
trained_model_names = models.keys()

model_scores = [models[model].test_score for model in trained_model_names]

max_score = max(model_scores)

best_model = trained_model_names[model_scores.index(max_score)]

print best_model,max_score,'\n\n',models[best_model].classification_report

Logistic regression with with scaling and selection appears to outperform all other scenarios so far.

Let's look at the best model's properties:

In [ ]:
print 'Best model:','logistic_regression','\n'
print 'Training score:\t\t', models['logistic_regression'].train_score
print 'Test score:\t\t', models['logistic_regression'].test_score,'\n'
print models['logistic_regression'].classification_report

print 'Confusion matrix:\n\n',models['logistic_regression'].confusion_matrix,'\n'
print 'Normalized confusion matrix:\n\n',models['logistic_regression'].normalized_confusion_matrix,'\n'
print 'Best parameters:\n', models['logistic_regression'].best_parameters

In [ ]:
dir(models['logistic_regression'].steps[0][1])

In [ ]:
import numpy as np
np.logspace(-10,10,100)

In [ ]:
%%time 
reload(dsl)
        
param_dist = {
    'estimator__C': np.logspace(-10,10,100),
    'estimator__penalty': ['l2','l1']
}
    
# Figure out best model
models['custom_logistic_regression'] = dsl.train_model(X,y,
                                use_default_param_dist=True,
                                random_state=6,
                                suppress_output=False, # Can suppress print outs if desired
                                estimator='logistic_regression',
                               param_dist=param_dist) 

In [ ]:
model_coefficients = models['custom_logistic_regression'].steps[0][1].coef_[0]

feature_ind = 0
indices = []
for feature in simulation_df.columns:
    if feature != 'Survived_1':
        print feature, '\t',model_coefficients[feature_ind]
        
        feature_ind += 1
        
model_df = pd.Series(model_coefficients,index=[feature for feature in simulation_df.columns if feature != 'Survived_1'])

In [ ]:
models.keys()

In [ ]:
model_df.sort_values()

Looks like being a Reverend had a very negative effect on survival.

In [ ]:
df['Survived'][df['Title']=='Dr'].value_counts()

There were six reverends and all died.

Next worst was having the title of "Mr". This is in contrast to just generally being a male.

In [ ]:
df[df['Title']=='Noble']

In [ ]:
df[(df['Sex']=='male')&(~df['Title'].isin(['Mr','Dr','Rev']))]

In [ ]:
simulation_df[simulation_df['Title_Military']==1]

In [ ]:
print models['select_scaled_logistic_regression'].steps[2][1].coef_[0].shape
print X.shape

In [ ]:
feature_mask = models['select_scaled_logistic_regression'].steps[0][1].get_support()
input_features = [feature for feature in simulation_df.columns if feature != 'Survived_1']

model_coefficients = models['select_scaled_logistic_regression'].steps[2][1].coef_[0]

feature_ind = 0
for feature in np.array(input_features)[feature_mask]:
    print feature, '\t',model_coefficients[feature_ind]
        
    feature_ind += 1

In [ ]:
for x in models['select_scaled_logistic_regression'].steps[2][1].coef_[0]:
    print x

print simulation_df.columns

In [ ]:
model_coefficients = models['select_scaled_logistic_regression'].steps[2][1].coef_[0]

feature_ind = 0
for feature in simulation_df.columns:
    if feature != 'Survived_1': 
        print feature, '\t',model_coefficients[feature_ind]
        
        feature_ind += 1
        
print model_coefficients[feature_ind]

In [ ]:
simulation_df[simulation_df['Title_Dr']==1]